### Longitudinal cortical thickness and fixel based analysis in Visual Hallucinations

- Author: A.Zarkali
- Date last updated: 1/1/2021
- Aim: Perform all demographics and tract comparisons for longitudinal VIPD data

### Load necessary libraries and data

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import shapiro
from statsmodels.formula.api import ols
import scikit_posthocs as sp
from pandas.api.types import is_numeric_dtype
import numpy as np
# Enable inline plotting
%matplotlib inline

In [2]:
# Load database of all demographics
df1 = pd.read_excel(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/Visit1Data.xlsx")
df2 = pd.read_excel(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/\Visit2Data.xlsx")
df = pd.concat((df1, df2), axis=1)

In [36]:
# Load Thickness data
thick1 = pd.read_table(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/Session1Aseg.txt")
thick2 = pd.read_table(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\02_StructuralMRI\02_ThalamicSegmentation\Session2Aseg.txt")
# thickCortex1 = pd.read_csv(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\02_StructuralMRI\02_ThalamicSegmentation\Session1Thickness.csv")
# thickCortex2 = pd.read_csv(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\02_StructuralMRI\02_ThalamicSegmentation\Session2Thickness.csv")

In [3]:
# Load thalamic data
thalVis1 = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/ThalamSegmVH_Visit1.csv", sep=",")
thalVis2 = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/\ThalamSegmVH_Visit2.csv", sep=",")
thalDif = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/\ThalamusDif.csv", sep=",")
# Load tract data
fcVis1 = pd.read_table(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/MRI_DATA/Mean_ThalamicTracts/IndividualNuclei/Baseline_fc_Thresholded.txt")
fcDif = pd.read_table(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/MRI_DATA/Mean_ThalamicTracts/IndividualNuclei/VisitDif_fc_Thresholded.txt")
fcVis2 = fcVis1 + fcDif

In [47]:
#### Extract FS mean TIV
participants = [df.Participant.values][0]
indeces = []
for i in range(len(participants)):
    for k in range(len(thick1)):
        if participants[i] in thick1["Measure:volume"].iloc[k]:
    #thick1["Measure:volume"].iloc[i] in participants:
            indeces.append(i)
thick1.EstimatedTotalIntraCranialVol[indeces].to_csv("ETIV_Session1.csv")

In [62]:
# Correct the thalamic volumes by ETIV
for i in range(len(thalVis2)):
    for col in thalVis2.drop(columns=["Participant"]).columns:
        thalVis2[col].iloc[i] = thalVis2[col].iloc[i] / df.EstimatedTotalIntraCranialVol.iloc[i]
thalVis2.to_csv("ThalamicVolumesPercentageTIV_Session2.csv")

In [63]:
# Combine clinical and thalamic segmentation data to a single dataframe 
thalamusVis1 = pd.concat([thalVis1,df], axis=1)
thalamusVis2 = pd.concat([thalVis2,df], axis=1)
thalamusDif = pd.concat([thalDif,df], axis=1)
# Combine clinical and tract data to a single dataframe
tractVis1 = pd.concat([fcVis1, df], axis=1)
tractVis2 = pd.concat([fcVis2, df], axis=1)
tractDif = pd.concat([fcDif, df], axis=1)

In [64]:
#### Create a long dataframe format
#### Thalamus
thalVis1["Session"] = 1
thalVis2["Session"] = 2
demographics = ["Age", "Gender", "PD_VHAny", "PD", "IntracranialVolume", "Time2Scan"]
for col in demographics:
    thalVis1[col] = df[col]
    thalVis1["Time2Scan"] = 0
    thalVis2[col] = df[col]
longthal = pd.concat([thalVis1, thalVis2], axis=0)

#### Tracts
fcVis1["Session"] = 1
fcVis2["Session"] = 2
demographics = ["Age", "Gender", "PD_VHAny", "PD", "IntracranialVolume", "Time2Scan", "Participant"]
for col in demographics:
    fcVis1[col] = df[col]
    fcVis1["Time2Scan"] = 0
    fcVis2[col] = df[col]
longFC = pd.concat([fcVis1, fcVis2], axis=0)

### Group thalamic nuclei to categories

In [65]:
### Lists to Group thalamic nuclei according to Iglesias et al. A probabilistic atlas of the human thalamic nuclei combining ex vivo MRI and histology. Neuroimage 2018 Dec; 183: 314–326
LeftAnterior = ["LeftAV"]
RightAnterior = ["RightAV"]
LeftLateral = ["LeftLD", "LeftLP"]
RightLateral = ["RightLD", "RightLP"]
LeftVentral = ["LeftVA", "LeftVAmc", "LeftVLa", "LeftVLp", "LeftVPL", "LeftVM"]
RightVentral = ["RightVA", "RightVAmc", "RightVLa", "RightVLp", "RightVPL", "RightVM"]
LeftIntralaminar = ["LeftCeM", "LeftCL", "LeftPc", "LeftCM", "LeftPf"]
RightIntralaminar = ["RightCeM", "RightCL", "RightPc", "RightCM", "RightPf"]
LeftMedial = ["LeftPt", "LeftMV_Re", "LeftMDm", "LeftMDl"]
RightMedial = ["RightPt", "RightMV-Re", "RightMDm", "RightMDl"]
LeftMedialGN = ["LeftMGN", "LeftL_SG"]
RightMedialGN = ["RightMGN", "RightL_SG"]
LeftPulvinar = ["LeftPuA", "LeftPuM", "LeftPuL", "LeftPuI"]
RightPulvinar = ["RightPuA", "RightPuM", "RightPuL", "RightPuI"]
LeftPosterior = LeftMedialGN + LeftPulvinar
RightPosterior = RightMedialGN + RightPulvinar
## List of the columns
SumColumns = ["LeftAnterior", "RightAnterior", "LeftLateral", "RightLateral", "LeftVentral", "RightVentral", "LeftIntralaminar", "RightIntralaminar", "LeftMedial", "RightMedial", "LeftPulvinar", "RightPulvinar", "LeftMedialGN", "RightMedialGN"]

In [66]:
### Create Columns of sum thalamic volumes per nuclei category
databases = thalamusVis1, thalamusVis2, thalamusDif
for base in databases:
    base["LeftAnterior"] = base["LeftAV"]
    base["RightAnterior"] = base["RightAV"]
    base["LeftLateral"] = base["LeftLD"] + base["LeftLP"]
    base["RightLateral"] = base["RightLD"] + base["RightLP"]
    base["LeftVentral"] = base["LeftVA"] + base["LeftVAmc"] + base["LeftVLa"] + base["LeftVLp"] + base["LeftVPL"] + base["LeftVM"]
    base["RightVentral"] = base["RightVA"] + base["RightVAmc"] + base["RightVLa"] + base["RightVLp"] + base["RightVPL"] + base["RightVM"]
    base["LeftIntralaminar"] = base["LeftCeM"] + base["LeftCL"] + base["LeftPc"] + base["LeftCM"] + base["LeftPf"]
    base["RightIntralaminar"] = base["RightCeM"] + base["RightCL"] + base["RightPc"] + base["RightCM"] + base["RightPf"]
    #base["LeftMedial"] = base["LeftMDl"] + ["LeftMDm"] + base["LeftPt"] + base["LeftMV_Re"] 
    #base["RightMedial"] = base["RightPt"] + base["RightMV_Re"] + ["RightMDm"] + base["RightMDl"]
    base["LeftMedialGN"] = base["LeftMGN"] + base["LeftL_Sg"]
    base["RightMedialGN"] = base["RightMGN"] + base["RightL_Sg"]
    base["LeftPulvinar"] = base["LeftPuA"] + base["LeftPuM"] + base["LeftPuL"] + base["LeftPuI"]
    base["RightPulvinar"] = base["RightPuA"] + base["RightPuM"] + base["RightPuL"] + base["RightPuI"]

In [67]:
### Medial one - loop doesnt work
thalamusVis1["LeftMedial"] = thalamusVis1.LeftMDl + thalamusVis1.LeftMDm + thalamusVis1.LeftPt + thalamusVis1.LeftMV_Re
thalamusVis1["RightMedial"] = thalamusVis1.RightMDl + thalamusVis1.RightMDm + thalamusVis1.RightPt + thalamusVis1.RightMV_Re
thalamusVis2["LeftMedial"] = thalamusVis2.LeftMDl + thalamusVis2.LeftMDm + thalamusVis2.LeftPt + thalamusVis2.LeftMV_Re
thalamusVis2["RightMedial"] = thalamusVis2.RightMDl + thalamusVis2.RightMDm + thalamusVis2.RightPt + thalamusVis2.RightMV_Re
thalamusDif["LeftMedial"] = thalamusDif.LeftMDl + thalamusDif.LeftMDm + thalamusDif.LeftPt + thalamusDif.LeftMV_Re
thalamusDif["RightMedial"] = thalamusDif.RightMDl + thalamusDif.RightMDm + thalamusDif.RightPt + thalamusDif.RightMV_Re

### Check for normality

In [23]:
## Normality check for all columns together

### Declare empty variables to hold column names
NormallyDistributed = []
NonNormallyDistributed = []

### Loop through all columns
for col in df.columns:
    if is_numeric_dtype(df[col]) == True: ## Numeric check
        data = df[np.isfinite(df[col])] ## Drop NAs (the shapiro will not calculate statistic if NAs present)
        r, p = stats.shapiro(data[col]) ### If less than 0.05 non normally distributed
        if p < 0.05: 
            NonNormallyDistributed.append(col)
        else:
            NormallyDistributed.append(col)
### Save in text files the names of the variables
with open('NormallyDistributedVariables.txt', 'w') as filehandle:
    for listitem in NormallyDistributed:
        filehandle.write('%s\n' % listitem)
with open('NonNormallyDistributedVariables.txt', 'w') as filehandle:
    for listitem in NonNormallyDistributed:
        filehandle.write('%s\n' % listitem)

In [16]:
#### Visualise distribution - if want to for individual columns
col = "HADSdepression"
sns.distplot(df[df.PD==1][col])
stats.shapiro(df[df.PD==1][col])

### Demographics

##### Extract group means and std

In [39]:
## Save all group mean and std for each column
### Group by PD_VHAny: 0 controls, 1 PD non VH, 2 PD VH
dfMean = df.groupby("PD_VHAny").mean()
dfMean["Type"] = "Mean"
dfSTD = df.groupby("PD_VHAny").std()
dfSTD["Type"] = "STD"
pd.concat([dfMean,dfSTD], axis=0).to_csv("GroupedDemographics.csv")

##### Comparisons between 3 groups


- Step 1 - Loop through all variables and run ANOVA for normally distributed and Kruskal Wallis for non normally distributed ones
- Step 2 - Post hoc testing for those who are significantly different

In [18]:
def groupCompare(variables, group, dataframe, number_groups): 
    ### Declare empty variables to hold column names
    NormallyDistributed = []
    NonNormallyDistributed = []
    statistic = []
    p_value = []
    types = []
    ### Loop through all columns of a dataframe and check normality
    for col in dataframe.columns:
        if is_numeric_dtype(dataframe[col]) == True: ## Numeric check
            data = dataframe[np.isfinite(dataframe[col])] ## Drop NAs (the shapiro will not calculate statistic if NAs present)
            r, p = stats.shapiro(data[col]) ### If less than 0.05 non normally distributed
            if p < 0.05: 
                NonNormallyDistributed.append(col)
            else:
                NormallyDistributed.append(col)
    for var in variables:
        if number_groups > 2: 
            if var in NormallyDistributed: ## Normally distributed then do ANOVA
                data=dataframe[np.isfinite(dataframe[var])]
                variable = data[var].dropna()
                comp = data[group] ### comparison of interest
                anova = ols("variable ~ C(comp)", data=data).fit() ### run anova  
                r = anova.rsquared_adj ## extract overall model adjusted r statistic
                p = anova.f_pvalue ## extract overall model p-value
                statistic.append(r)
                p_value.append(p)
                types.append("ANOVA")
            elif var in NonNormallyDistributed: ### Non normally distributed then do Kruskal Wallis
                data = dataframe[np.isfinite(dataframe[var])] 
                ### declare the three series
                v1 = data[data[group] == 0][var] 
                v2 = data[data[group] == 1][var]
                v3 = data[data[group] == 2][var]
                r,p = stats.kruskal(v1, v2, v3) ### run Kruskal wallis
                statistic.append(r)
                p_value.append(p)
                types.append("Kruskal-Wallis")
            else: ### In case any variables were labelled incorrectly
                statistic.append("NA")
                p_value.append("NA")
                types.append("NA")
        elif number_groups == 2:
            if var in NormallyDistributed: ## Normally distributed then do ttest
                data=dataframe[np.isfinite(dataframe[var])]
                v1 = data[data.PD_VHAny == 1][var]
                v2 = data[data.PD_VHAny == 2][var]
                r, p = stats.ttest_ind(v1, v2)
                statistic.append(r)
                p_value.append(p)
                types.append("t-test")
            elif var in NonNormallyDistributed: ### Non normally distributed then do Mann-Whitney
                data = dataframe[np.isfinite(dataframe[var])] 
                v1 = data[data.PD_VHAny == 1][var]
                v2 = data[data.PD_VHAny == 2][var]
                r,p = stats.mannwhitneyu(v1, v2) ### run Kruskal wallis
                statistic.append(r)
                p_value.append(p)
                types.append("Mann-Whitney")
            else: ### In case any variables were labelled incorrectly
                statistic.append("NA")
                p_value.append("NA")
                types.append("NA")    
    ### Combine results on dataframe
    results = pd.DataFrame(data=np.zeros((len(variables), 0))) # empty dataframe
    results["Variable"] = variables # variable names
    results["Statistic"] = statistic # statistic
    results["Pvalue"] = p_value # p_value
    results["Type"] = types # type of statistical test used
    return(results)

In [15]:
filename = r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\02_StructuralMRI\02_ThalamicSegmentation\DemographicsAll3groups.txt"
variables = [line.rstrip('\n') for line in open(filename)]
result = groupCompare(variables,"PD_VHAny",df,2)

In [174]:
## Perform comparisons between all 3 groups at once
### Load variables for 3 group comparison
filename = r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\02_StructuralMRI\02_ThalamicSegmentation\DemographicsAll3groups.txt"
variables = [line.rstrip('\n') for line in open(filename)]

## Empty variables to hold results
statistic = []
p_value = []
types = []

### Loop through all the variables
for var in variables:
    if var in NormallyDistributed: ## Normally distributed then do ANOVA
        data=df[np.isfinite(df[var])]
        variable = data[var].dropna()
        group = data.PD_VHAny ### comparison of interest
        anova = ols("variable ~ C(group)", data=data).fit() ### run anova  
        r = anova.rsquared_adj ## extract overall model adjusted r statistic
        p = anova.f_pvalue ## extract overall model p-value
        statistic.append(r)
        p_value.append(p)
        types.append("ANOVA")
    elif var in NonNormallyDistributed: ### Non normally distributed then do Kruskal Wallis
        data = df[np.isfinite(df[var])] 
        ### declare the three series
        v1 = data[data.PD_VHAny == 0][var] 
        v2 = data[data.PD_VHAny == 1][var]
        v3 = data[data.PD_VHAny == 2][var]
        r,p = stats.kruskal(v1, v2, v3) ### run Kruskal wallis
        statistic.append(r)
        p_value.append(p)
        types.append("Kruskal-Wallis")
    else: ### In case any variables were labelled incorrectly
        statistic.append("NA")
        p_value.append("NA")
        types.append("NA")
### Combine results on dataframe
results = pd.DataFrame(data=np.zeros((len(variables), 0))) # empty dataframe
results["Variable"] = variables # variable names
results["Statistic"] = statistic # statistic
results["Pvalue"] = p_value # p_value
results["Type"] = types # type of statistical test used
results.to_csv("GroupComparisons_3Groups.csv") # export to csv

In [14]:
### Post hoc testing
# Variables that are significantly different between groups
results[results.Pvalue <=0.05]

In [12]:
# Post hoc tukey test for ANOVA
var = "Stroop_colour_time"
data = df[np.isfinite(df[var])]
variable = data[var] # substitute variable sequentially to test all continuous variables following ANOVA
group = data.PD_VHAny
sp.posthoc_tukey_hsd(variable, group, alpha=0.05)
# The contrast appearing as 1 is significant / 0 is non significant / -1 for diagonals

In [11]:
## Post hoc dunn test for Kruskal Wallis
var = "Stroop_both_time"
X = [df[df.PD_VHAny == 0][var], df[df.PD_VHAny == 1][var], df[df.PD_VHAny == 2][var]]
sp.posthoc_dunn(X) 
### returns the exact p-values for each comparison

#### Comparisons between PD groups only

In [106]:
### Continuous variables
### Load variables for PD group comparison
filename = r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\02_StructuralMRI\02_ThalamicSegmentation\DemographicsPDgroups.txt"
variables = [line.rstrip('\n') for line in open(filename)]
## Empty variables to hold results
statistic = []
p_value = []
types = []

### Loop through all the variables
for var in variables:
    if var in NormallyDistributed: ## Normally distributed then do ANOVA
        data=df[np.isfinite(df[var])]
        v1 = data[data.PD_VHAny == 1][var]
        v2 = data[data.PD_VHAny == 2][var]
        r, p = stats.ttest_ind(v1, v2)
        statistic.append(r)
        p_value.append(p)
        types.append("t-test")
    elif var in NonNormallyDistributed: ### Non normally distributed then do Kruskal Wallis
        data = df[np.isfinite(df[var])] 
        v1 = data[data.PD_VHAny == 1][var]
        v2 = data[data.PD_VHAny == 2][var]
        r,p = stats.mannwhitneyu(v1, v2) ### run Kruskal wallis
        statistic.append(r)
        p_value.append(p)
        types.append("Mann-Whitney")
    else: ### In case any variables were labelled incorrectly
        statistic.append("NA")
        p_value.append("NA")
        types.append("NA")
### Combine results on dataframe
results = pd.DataFrame(data=np.zeros((len(variables), 0))) # empty dataframe
results["Variable"] = variables # variable names
results["Statistic"] = statistic # statistic
results["Pvalue"] = p_value # p_value
results["Type"] = types # type of statistical test used
results.to_csv("GroupComparisons_PDonly.csv") # export to csv

In [121]:
### Categorical variables
filename = r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/02_StructuralMRI/02_ThalamicSegmentation/DemographicsPDgroupsCategorical.txt"
variables = [line.rstrip('\n') for line in open(filename)]

## Empty variables to hold results
statistic = []
p_value = []

for var in variables:
    data=df[np.isfinite(df[var])]
#     data = data[data.PD==1] ## do only in PD
    tab = pd.crosstab(data.PD_VHAny,data[var])
    r, p, dof, exp = stats.chi2_contingency(tab)
    statistic.append(r)
    p_value.append(p)
### Combine results on dataframe
results = pd.DataFrame(data=np.zeros((len(variables), 0))) # empty dataframe
results["Variable"] = variables # variable names
results["ChiSquare"] = statistic # statistic
results["Pvalue"] = p_value # p_value
results.to_csv("GroupComparisons_PDonlyCategorical.csv") # export to csv

In [131]:
### Comparison of longitudinal difference
variablesVisit1 = ["MOCA", "MMSE", "DigitSpanF", "DigitSpanB", "Stroop_colour_time", "Stroop_both_time", "FluencyAnimal", "WordRec", "LogMem_Delayed", "GNT", "FluencyLetter", "JLO", "Hooper", "UPDRS", "UPDRSMotorScoreonly", "LEDD"]
variablesVisit2 = ["MOCA_Session2", "MMSE_Session2", "Digit_Span_Forward_Session2", "Digit_Span_Backward_Session2", "Stroop_colour_time_Session2", "Stroop_both_time_Session2", "Verbal_fluency_category_Session2", "Word_recognition_Session2", "Logical_Memory_Delayed_Session2", "GNT_Session2", "Verbal_fluency_letter_Session2", "JLO_Session2", "Hooper_Session2", "UPDRS_Session2", "UPDRSMotorScoreOnly_Session2", "LEDD_S2"]

variablesDif = []

dfPD = df[df.PD ==1]
for x in range(len(variablesVisit1)):
    variable = str(variablesVisit1[x] + "_Dif")
    variablesDif.append(variable)
    dfPD[variable] = dfPD[(variablesVisit2[x])] - dfPD[(variablesVisit1[x])] 
resultsDif = groupCompare(variablesDif, "PD_VHAny", dfPD, 2)
resultsDif.to_csv("GroupDifferences_LongitudinalCognition_PDonly.csv")

### Compare across groups

#### 1. Compare individual nuclei or tracts

In [26]:
#### Loop through all the individual nuclei
## Declare the datase
data = tractVis1
#data = data[data.PD == 1]  ## Select PD only
#data = data[data.PD_VHAny!=1] ## Remove PD non VH
data["groups"] = 0 
vc = {"IntracranialVolume": "IntracranialVolume", "Gender": "0 + C(Gender)", "Age": "Age"} 
# # Declare empty lists
pvalues = []
coefficients = []
lowerCI = []
upperCI = []

### Loop through all nuclei 
#for nucleus in thalVis1.drop(columns=["Participant", "LeftWhole_thalamus", "RightWhole_thalamus"]).columns: ### for nuclei
for nucleus in fcVis1.columns: ### for tracts
    # Change the Y variable to each ROI name
    formula = nucleus + " ~ PD"
    md = sm.MixedLM.from_formula(formula, data=data, re_formula="1", vc_formula=vc, groups=data.groups)
    mdf = md.fit() # fit the model
    ### Select the values I want from the model
    p = mdf.pvalues[1]
    coef = (mdf.conf_int().iloc[1]).mean()
    lower = (mdf.conf_int().iloc[1])[0]
    upper = (mdf.conf_int().iloc[1])[1]
    ### Append them to the empty lists
    pvalues.append(p)
    coefficients.append(coef)
    lowerCI.append(lower)
    upperCI.append(upper)
# FDR correct the p-values
FDR = sm.stats.multipletests(pvalues, is_sorted=False, alpha=0.05, method="fdr_bh", returnsorted=False)

# Merge to Dataframe and export as csv
# outdata = pd.DataFrame(data=np.zeros(((len(thalVis1.columns)-3),0))) ### for nuclei
# outdata["Nucleus"] = thalVis1.drop(columns=["Participant", "LeftWhole_thalamus", "RightWhole_thalamus"]).columns ### For nuclei
outdata = pd.DataFrame(data=np.zeros((len(fcVis1.columns),0))) ### for tracts
outdata["Tract"] = fcVis1.columns ### for tracts
outdata["Coef"] = coefficients
outdata["lowerCI"] = lowerCI
outdata["upperCI"] = upperCI
outdata["pValues"] = pvalues
outdata["FDR"] = FDR[1]
outdata.to_csv("FCVis1_PDvsControls.csv")

In [70]:
##### LONGITUDINAL

#### Loop through all the individual nuclei
## Declare the datase
data = thalamusDif
data = data[data.PD == 1]
# data = data[data.PD_VHAny!=1]
data["groups"] = 0 

# # Declare empty lists
pvalues = []
coefficients = []
lowerCI = []
upperCI = []

### Loop through all nuclei 
for nucleus in thalVis1.drop(columns=["Participant", "LeftWhole_thalamus", "RightWhole_thalamus"]).columns: ### for nuclei
#for nucleus in fcVis1.columns: ### for tracts
    vis1volume = str(nucleus + "Vis_1")
    data[vis1volume] = thalVis1[nucleus]
    # Change the Y variable to each ROI name
    vc = {"IntracranialVolume": "IntracranialVolume", "Gender": "0 + C(Gender)", "Age": "Age", "Visit1":vis1volume} 
    formula = nucleus + " ~ C(PD_VHAny)* Time2Scan"
#     formula = nucleus + " ~ C(PD_VHAny) * Time + " + vis1volume
    md = sm.MixedLM.from_formula(formula, data=data, re_formula="1", vc_formula=vc, groups=data.groups)
    mdf = md.fit() # fit the model
    ### Select the values I want from the model
    p = mdf.pvalues[3]
    coef = (mdf.conf_int().iloc[3]).mean()
    lower = (mdf.conf_int().iloc[3])[0]
    upper = (mdf.conf_int().iloc[3])[1]
    ### Append them to the empty lists
    pvalues.append(p)
    coefficients.append(coef)
    lowerCI.append(lower)
    upperCI.append(upper)
# FDR correct the p-values
FDR = sm.stats.multipletests(pvalues, is_sorted=False, alpha=0.05, method="fdr_bh", returnsorted=False)

# Merge to Dataframe and export as csv
outdata = pd.DataFrame(data=np.zeros(((len(thalVis1.columns)-3),0))) ### for nuclei
outdata["Nucleus"] = thalVis1.drop(columns=["Participant", "LeftWhole_thalamus", "RightWhole_thalamus"]).columns ### For nuclei
#outdata = pd.DataFrame(data=np.zeros((len(fcVis1.columns),0))) ### for tracts
#outdata["Tract"] = fcVis1.columns ### for tracts
outdata["Coef"] = coefficients
outdata["lowerCI"] = lowerCI
outdata["upperCI"] = upperCI
outdata["pValues"] = pvalues
outdata["FDR"] = FDR[1]
outdata.to_csv("Thalamus_LongitudinalPD_VHAny.csv")

In [102]:
#Individual ones that fail
data = tractVis1
# data = data[data.PD == 1]
data = data[data.PD_VHAny != 1]
data["groups"] = 0 
vc = {"IntracranialVolume": "IntracranialVolume", "Age": "Age"} 
formula = "RightPuM" + " ~ PD_VHAny"
md = sm.MixedLM.from_formula(formula, data=data, re_formula="1", vc_formula=vc, groups=data.groups)
mdf = md.fit()
mdf.summary()

In [1]:
#### LONGITUDINAL LME4
import os
os.environ["R_HOME"] = r"C:/PROGRA~1/R/R-3.6.1"
os.environ["PATH"]   = r"C:/PROGRA~1/R/R-3.6.1\bin\x64" + ";" + os.environ["PATH"]
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from pymer4.models import Lmer


# Clear longitudinal data
data = longthal
#data = data[data.PD == 1] ## PD only
#data = data[data.PD_VHAny !=1] ## Drop PD non VH
data["subject"] = data.Participant.str.slice(start=-3).astype(int) ### Add a subject column as ints
dropColumns = demographics + ["Participant", "LeftWhole_thalamus", "RightWhole_thalamus", "Session"] ## drop columns to get only the thalami 
#dropColumns = demographics + ["Session"] ## for Tracts

# # Declare empty lists
pvalues = []
T_stats = []
coefficients = []
lowerCI = []
upperCI = []

# nuclei = thalVis1.drop(columns="Participant").columns
### Loop through all nuclei 
for nucleus in thalVis1.drop(columns=dropColumns).columns: ### for nuclei
    formula = str(nucleus + " ~ PD * Time2Scan + Age + Gender + IntracranialVolume  + (1 | subject)")
    model = Lmer(formula, data=data)
    m = model.fit()
    # get the values for the Slope
    mdf = m.loc["PD:Time2Scan"]
    coef = mdf["Estimate"]
    lower = mdf["2.5_ci"]
    upper = mdf["97.5_ci"]
    p = mdf["P-val"]
    t = mdf["T-stat"]
    ### Append them to the empty lists
    pvalues.append(p)
    coefficients.append(coef)
    lowerCI.append(lower)
    upperCI.append(upper)
    T_stats.append(t)
    
# FDR correct the p-values
FDR = sm.stats.multipletests(pvalues, is_sorted=False, alpha=0.05, method="fdr_bh", returnsorted=False)

# Merge to Dataframe and export as csv
outdata = pd.DataFrame(data=np.zeros((len(thalVis1.drop(columns=dropColumns).columns),0))) ### for nuclei
outdata["Nucleus"] = thalVis1.drop(columns=dropColumns).columns ### For nuclei
# outdata["Tract"] = fcVis1.drop(columns=dropColumns).columns ### for tracts
outdata["T-stat"] = T_stats
outdata["Coef"] = coefficients
outdata["lowerCI"] = lowerCI
outdata["upperCI"] = upperCI
outdata["pValues"] = pvalues
outdata["FDR"] = FDR[1]
outdata.to_csv("Thalami_LongitudinalPD.csv")

In [2]:
df.groupby("PD_VisPerf").Hallucinations_Session2.mean(), df.groupby("PD_VisPerf").Hallucinations_Session2.std()

In [3]:
v1 = df[df1.PD_VisPerf == 1].Hallucinations_Session2
v2 = df[df1.PD_VisPerf == 2].Hallucinations_Session2
data = df[df.PD_VisPerf != 0]
tab = pd.crosstab(data.PD_VisPerf, data.PD_VHAny)
stats.chi2_contingency(tab)

#### 2. Compare categories of nuclei

In [35]:
### Loop through all the nuclei categories
## Declare the datase
data = tractDif
data = data[data.PD == 1]
data["groups"] = 0 
#vc = {"IntracranialVolume": "IntracranialVolume", "Age": "Age"}
vc = {"IntracranialVolume": "IntracranialVolume", "Gender": "0 + C(Gender)", "Age": "Age"} 
# # Declare empty lists
pvalues = []
coefficients = []
lowerCI = []
upperCI = []

### Loop through all nuclei 
for nucleus in fcVis1.columns:
    # Change the Y variable to each ROI name
    formula = nucleus + " ~ VH_Any"
    md = sm.MixedLM.from_formula(formula, data=data, re_formula="1", vc_formula=vc, groups=data.groups)
    mdf = md.fit()
    p = mdf.pvalues[1]
#     coef = (mdf.conf_int().loc["PD"]).mean()
#     lower = (mdf.conf_int().loc["PD"])[0]
#     upper = (mdf.conf_int().loc["PD"])[1]
    coef = (mdf.conf_int().iloc[1]).mean()
    lower = (mdf.conf_int().iloc[1])[0]
    upper = (mdf.conf_int().iloc[1])[1]
    pvalues.append(p)
    coefficients.append(coef)
    lowerCI.append(lower)
    upperCI.append(upper)
# FDR correct
FDR = sm.stats.multipletests(pvalues, is_sorted=False, alpha=0.05, method="fdr_bh", returnsorted=False)

# Merge to Dataframe and export as csv
outdata = pd.DataFrame(data=np.zeros((len(fcVis1.columns),0))) 
outdata["Nucleus"] = fcVis1.columns
outdata["Coef"] = coefficients
outdata["lowerCI"] = lowerCI
outdata["upperCI"] = upperCI
outdata["pValues"] = pvalues
outdata["FDR"] = FDR[1]
outdata.to_csv("VisitDifThalamus_VHvsNonVH_Categories.csv")

#### 3. Compare Whole thalamus

In [204]:
### Repeat with whole thalamus
data = thalamusDif
data = data[data.PD_VHAny != 1]
data["groups"] = 0
data["Whole_thalamus"] = data.LeftWhole_thalamus + data.RightWhole_thalamus
md = sm.MixedLM.from_formula("Whole_thalamus ~ VH_Any", data=data, re_formula="1", vc_formula=vc, groups=data.groups)
mdf = md.fit()
mdf.summary()

### Visualisations

##### Individual Nuclei Barplots

In [200]:
### Barplots and swarmplot with all thalamic volumes between Controls, PD-VH, PD-nonVH
### The database you want to loop through (aka Baseline Visit, Visit 2 etc)
data = thalamusVis1

### Set the style parameters for the graph
colors= ["grey", "salmon", "brown"] # colour dictionary
sns.set_palette(sns.color_palette(colors)) 
sns.set_style("white")
sns.set_context("poster", rc={"font.size":18,"axes.titlesize":18,"axes.labelsize":18})   

#### Loop through all the nuclei
#for nucleus in thalVis1.drop(columns=["Participant", "LeftWhole_thalamus", "RightWhole_thalamus"]).columns: ## If looping through individual nuclei
for nucleus in SumColumns: ### If looping through categories
    fig, ax = plt.subplots(figsize=(10,10)) # set size
    sns.boxplot(x="PD_VHAny", y=nucleus, data=data) # boxplot
    sns.swarmplot(x="PD_VHAny", y=nucleus, data=data, color="black", size=6) # superimposed swarmplot
    # filename to save output - this needs to be unique
    outname = str(nucleus + "_Visit1_Barplot.png") 
    # Make labels pretty
    ax.set_xticklabels(["Controls", "PD non VH", "PD VH"], fontsize=20)
    plt.xlabel(" ")
    plt.ylabel(nucleus, fontsize=20)
    # Save file, set dpi to 300
    plt.savefig(outname, dpi=300)

In [4]:
### Create longitudinal plots for each nucleus
# Set style for the graphs
colors= ["salmon", "brown"]
palette=sns.color_palette(colors)
sns.set_style("white")

nuc = ["RightMDm", "LeftPc"]
## Loop through all nuclei
for nucleus in nuc:
#for nucleus in thalVis1.drop(columns=["Participant", "LeftWhole_thalamus", "RightWhole_thalamus"]).columns: ## If looping through individual nuclei
#for nucleus in SumColumns: ### If looping through categories
    # Clear the data 
    #### Need to select the specific nucleus I want from visit 1 and visit 2 and then melt the database
    data = thalVis1[["Participant",nucleus]]
    data[str(nucleus + "_S2")] = thalVis2[nucleus]
    data["PD_VHAny"] = df.PD_VHAny ### This is the group of interest
    data = data[data.PD_VHAny !=0]
    dataMelt = data.melt(id_vars="PD_VHAny", value_vars=[nucleus, (str(nucleus + "_S2"))])
    #### Initiate graph pointplot, errorbars:95CI
    fig, ax = plt.subplots(figsize=(10,10))
    fig = sns.pointplot(y="value", x="variable",hue="PD_VHAny", data=dataMelt, palette=palette, dodge=True, scale=0.5, errwidth=2, legend=True) 
    #### Make the labels pretty 
    ax.set_xticklabels(["Baseline", "Session 2"], fontsize=20)
    plt.yticks(fontsize=18)
    plt.xlabel(" ")
    plt.ylabel(nucleus, fontsize=20)
    ### Add legend with the correct colours
    l = plt.legend(title="", loc="upper right", labels= ["PD non VH", "PD VH"], fontsize=16)
    l.legendHandles[0].set_color(colors[0])
    l.legendHandles[1].set_color(colors[1])
    #l.legendHandles[2].set_color(colors[2])
    ### Filename to save output
    outname = str(nucleus + "_Longitudinal.png")
    ### Save
    plt.savefig(outname, dpi=300)

In [24]:
## Create longitudinal plots for each thalamic tract
# Set style for the graphs
colors= ["grey", "salmon", "brown"]
palette=sns.color_palette(colors)
sns.set_style("white")

## Loop through all nuclei
for nucleus in fcVis1.columns: ## If looping through individual nuclei
#for nucleus in SumColumns: ### If looping through categories
    # Clear the data 
    #### Need to select the specific nucleus I want from visit 1 and visit 2 and then melt the database
    data = fcVis1[[nucleus]]
    data[str(nucleus + "_S2")] = fcVis2[nucleus]
    data["PD_VHAny"] = df.PD_VHAny ### This is the group of interest
    dataMelt = data.melt(id_vars="PD_VHAny", value_vars=[nucleus, (str(nucleus + "_S2"))])
    #### Initiate graph pointplot, errorbars:95CI
    fig, ax = plt.subplots(figsize=(10,10))
    fig = sns.pointplot(y="value", x="variable",hue="PD_VHAny", data=dataMelt, palette=palette, dodge=True, scale=0.5, errwidth=2, legend=True) 
    #### Make the labels pretty 
    ax.set_xticklabels(["Baseline", "Session 2"], fontsize=20)
    plt.yticks(fontsize=18)
    plt.xlabel(" ")
    plt.ylabel(nucleus, fontsize=20)
    ### Add legend with the correct colours
    l = plt.legend(title="", loc="upper right", labels= ["Controls", "PD non VH", "PD VH"], fontsize=16)
    l.legendHandles[0].set_color(colors[0])
    l.legendHandles[1].set_color(colors[1])
    l.legendHandles[2].set_color(colors[2])
    ### Filename to save output
    outname = str(nucleus + "_Longitudinal.png")
    ### Save
    plt.savefig(outname, dpi=300)

##### Miami and Nuclei correlation

In [5]:
#### Thalamic tracts
### Set Style
sns.set_style("white")
sns.set_context("poster", rc={"font.size":28,"axes.titlesize":18,"axes.labelsize":18})   
fig, ax = plt.subplots(figsize=(20,20))
colors = ["salmon", "firebrick"]
palette = sns.color_palette(colors)

cols = ["MiamiAny", "PD_VHAny", "Participant"]
exclude = ["RightVM", "LeftVM", "RightPt", "LeftPt", "RightPc", "LeftPc"]
for col in fcVis1.columns:
    if col not in exclude:
        cols.append(col)

        data = tractDif[tractDif.PD == 1]
data = pd.melt(data[cols], id_vars=["PD_VHAny", "Participant", "MiamiAny"])

sns.scatterplot (y="value", x="MiamiAny", data=data, hue="PD_VHAny", ax=ax, legend=False, palette=palette)
sns.regplot (y="value", x="MiamiAny", data=data, scatter=False, ax=ax, color="black", line_kws={"lw":2})

# Make labels pretty
plt.xlabel("UM-PDHQ score")
plt.ylabel("Thalamic tracts Mean FC change", fontsize=20)
stats.spearmanr(data.value, data.MiamiAny)

plt.savefig("FCtracts_Miami.png")

In [6]:
#### Significant Nuclei

#### Thalamic tracts
### Set Style
sns.set_style("white")
sns.set_context("poster", rc={"font.size":28,"axes.titlesize":18,"axes.labelsize":18})   
fig, ax = plt.subplots(figsize=(20,20))
colors = ["salmon", "firebrick"]
palette = sns.color_palette(colors)

cols = ["MiamiAny", "PD_VHAny", "RightMDm", "LeftPc"]

data = thalamusDif[thalamusDif.PD == 1]
data = pd.melt(data[cols], id_vars=["PD_VHAny", "MiamiAny"])
data = data[data.variable == "LeftPc"]

sns.scatterplot (y="value", x="MiamiAny", data=data, hue="PD_VHAny", ax=ax, legend=False, palette=palette)
sns.regplot (y="value", x="MiamiAny", data=data, scatter=False, ax=ax, color="black", line_kws={"lw":2})

# Make labels pretty
plt.xlabel("UM-PDHQ score")
plt.ylabel("LeftPc", fontsize=20)
stats.spearmanr(data.value, data.MiamiAny)

#plt.savefig("LeftPc.png")

##### Visualise as percentage change

In [293]:
###### Clean Data
data = fcVis2.copy()
data["PD_VHAny"] = df.PD_VHAny
## Separate Low and High vis datasets
dataHigh = data[data.PD_VHAny == 1]
dataLow = data[data.PD_VHAny == 2]
## One column for each nucleus
nuclei = []
for col in fcVis1.columns:
    nuclei.append(col)
## Declare empty lists to hold results
resHigh_mean = np.zeros(len(nuclei))
resHigh_lower = np.zeros(len(nuclei))
resHigh_upper = np.zeros(len(nuclei))
resLow_mean =  np.zeros(len(nuclei))
resLow_lower =  np.zeros(len(nuclei))
resLow_upper =  np.zeros(len(nuclei))
for i in range(len(nuclei)):
    ## High Vis
    meanHigh = dataHigh[nuclei[i]].mean()
    stdHigh = dataHigh[nuclei[i]].std()
    resHigh_mean[i] = meanHigh
    resHigh_lower[i] = meanHigh - (2*stdHigh)
    resHigh_upper[i] = meanHigh + (2*stdHigh)
    ## Low Vis
    meanLow = dataLow[nuclei[i]].mean()
    stdLow = dataLow[nuclei[i]].std()
    resLow_mean[i] = meanLow
    resLow_lower[i] = meanLow - (2*stdLow)
    resLow_upper[i] = meanLow + (2*stdLow)

resultsMean = pd.DataFrame(data = nuclei)
resultsUpper = pd.DataFrame(data = nuclei)
resultsLower = pd.DataFrame(data = nuclei)

resultsMean["LowVis"] = resLow_mean
resultsUpper["LowVis"] = resLow_upper
resultsLower["LowVis"] = resLow_lower
resultsMean["HighVis"] = resHigh_mean
resultsUpper["HighVis"] = resHigh_upper
resultsLower["HighVis"] = resHigh_lower

resultsMean.to_csv("MeanFC.csv")
resultsUpper.to_csv("UpperFC.csv")
resultsLower.to_csv("LowerFC.csv")

In [7]:
sns.set_style("ticks")
sns.set_context("poster", rc={"font.size":12,"axes.titlesize":12,"axes.labelsize":12})   
results = pd.read_csv(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\02_StructuralMRI\02_ThalamicSegmentation\Visit2_FC_Perc.csv")

### Select the Left or Right only
results = results[results["0"].str.contains("Right")]

colorsSig = ["lightgray", "brown", "royalblue"]

palette = sns.color_palette(colorsSig)
fig = sns.catplot(x="VH", y="0", data=results, hue="Significant", kind="box", orient="h", legend=False, fliersize=0, whis=0, linewidth=1, height=20, aspect=0.6, sharey=True, palette=palette)
fig.set_axis_labels("", "")

plt.xlim(-1,2)
#plt.xlim(-0.25,0)
fig.savefig("Visit2_TractFC_Right", dpi=300)

#### Extra for Hippocampal Volume - FBA correlation

In [8]:
df["HipL"] = thick1["Left-Hippocampus"]
df["HipR"] = thick1["Right-Hippocampus"]
df["WMVol"] = thick1["CerebralWhiteMatterVol"]
df["eTIV"] = thick1["EstimatedTotalIntraCranialVol"]
v1 = df[df.PD_VisPerf == 0].eTIV.dropna()
v2 = df[df.PD_VisPerf == 1].eTIV.dropna()
v3 = df[df.PD_VisPerf == 2].eTIV.dropna()
stats.kruskal(v1,v2,v3)

In [ ]:
## Post hoc dunn test for Kruskal Wallis
X = [v1,v2,v3]
sp.posthoc_dunn(X) 
### returns the exact p-values for each comparison

In [10]:
# ANOVA test:
data=df
#data["DigitSpanF_dif"] = data.Verbal_fluency_letter_Session2 - data.FluencyLetter
data = data[np.isfinite(data.eTIV)]
variable = data.eTIV.dropna()
group = data.PD_VisPerf

anova = ols("variable ~ C(group)", data=data).fit()  
anova.summary()

# Prob(F-statistic): overall p- value
# Intercept: first variable group (for example in this is BGC/Interhemispheric)
# R-squared: effect size